In [1]:
import numpy as np
import igl
import meshplot as mp

# Reading point cloud

In [2]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Setting up  the constraints

In [3]:


#Function that retrieves the index of the closest point to 'point' in 'points'
#Use KD-Tree algorithm to find the nearest neighbor
from scipy.spatial import KDTree 
tree = KDTree(pi,leafsize = pi.shape[0] +1)  # leafsize is the number of points at which to switch to brute force

def find_closed_point(point,points):
    dist , ind = tree.query([point],k = 1) #k is the number of neighbors to return , and point is the array of points to query
    return ind #Each entry gives the list of indices of neighbors of the corresponding point 

In [4]:
# Add here the code to generate the additional points and constraints

def Constraints(pi,n):
    size = pi.shape[0] #366 points
    constrained_pi , constrained_v  = np.zeros((size*3 , 3)) , np.zeros((size*3))
    print(constrained_pi)
    print(constrained_v)
    print(constrained_v.shape)
    for i in range(size):
        eps = 0.01 * igl.bounding_box_diagonal(pi)# Fix an eps value 
        eps_plus = eps
        eps_minus = eps
        #for each point in the point cloud , add a costraint of the form f(pi)= di = o
        constrained_pi[i]= pi[i]
        constrained_v[i] = 0
        #for each point , compute p_i+N = p_i + eps * n_i
        #where n_i is the normalized normal at p_i.
        constrained_pi[i+size] =  constrained_pi[i] + eps * n[i]
        #Check that p_i is the closest point to p_i+N
        if(find_closed_point(constrained_pi[i+size],pi)) !=i:
        #If not , halve eps and recompute p_i+N until this is the case
            eps_plus = eps/2
        constrained_pi[i+size] =  constrained_pi[i] + eps_plus * n[i]
        #Then, add another constraint equation f(p_i+N) = d_i+N = eps
        constrained_v[i+size] = eps_plus
        
        #Repeat the same process for eps_minus
        
        constrained_pi[i + 2*size] = constrained_pi[i] - eps * n[i]
        #Remember to check each time that p_i is the closest point to p_i+2N
        
        if(find_closed_point(constrained_pi[i+size*2], pi)) != i:
            eps_minus = eps/2
        
        constrained_pi[i+ 2*size] = constrained_pi[i] - eps * n[i]
        #add an equation f(P_i+2N) = d_i+2N = eps_minus
        constrained_v[i+2*size] = -eps_minus
    
    return constrained_pi,constrained_v

In [5]:
constrained_pi , constrained_v = Constraints(pi,ni)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. ... 0. 0. 0.]
(1098,)


In [6]:
#let's build a color map for a better visualization
color_map = np.zeros(constrained_pi.shape)
size = pi.shape[0]
color_map[0:size] = np.array([1,0,0])
color_map[size:size*2] = np.array([0,0,1])
color_map[size*2:size*3]= np.array([0,1,0])

In [7]:
mp.plot(constrained_pi,c = color_map,shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

# Create a grid sampling the 3D space

In [8]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# MLS function

In [9]:
# This is mockup code, generating the distance field from the surface of the unit sphere in analytic form
# The code is provided to show the desired visualization
# Use tet_grid to generate the grid appropriate to your data
# Generate function fx at all nodes of the grid by evaluating the MLS function

In [10]:
#Wendland weighs
def Wendland(r,h):
    if r < h:
        return ((1-r/h)**4)*(4 * r / h + 1)
    else:
        return 0

In [11]:
#Function that retrieves the indices  of all points in 'points' that are at distance less than h from point
def closest_points(point,points,h):
    res = [] #variable where we store the indices
    #component of the point we are considering
    x = point[0]
    y = point[1]
    z = point[2]
    
    # scan for every point
    
    for i in range(points.shape[0]):
        relx = points[i][0]
        rely = points[i][1]
        relz = points[i][2]
        
        #squared euclidean distance
        d = (relx - x)**2 + (rely - y)**2 + (relz - z)**2
        if d < h**2:
            res.append(i)
    
    
    return res

In [12]:
#Degree k = 0,1,2 polynomial basis function
def polyDegree(degree,x):
    
    basis = {
        0 : np.array([1],dtype = object),
        1 : np.array([1,x[0],x[1],x[2]],dtype = object),
        2 : np.array([1,x[0],x[1],x[2],
                    x[0] * x[1], x[0] * x[2], x[1] * x[2],
                    x[0]**2,x[1]**2,x[2]**2],dtype = object)
    }
    
    #choose the array based on the degree we got in input
    return basis.get(degree) 

In [13]:
#test
a = [1,2,3]
print(polyDegree(1,a))

[1 1 2 3]


In [14]:
#Parameters
bbox_min = np.min(pi,axis = 0) #take the min point with all its components
bbox_max = np.max(pi,axis = 0) # take the max point with all its components
bbox_diag = np.linalg.norm(bbox_max - bbox_min)
print(bbox_min)
n = 10

[-22.7 -63.7 -99. ]


In [15]:
#At a give point xi in x , it finds the optimal basis function coefficients(which will vary from point to point)
# and using these to combine the basis function values at xi

#xi is the point, wendlandRadius is the h in closest_points function,
def MLS(xi,wendlandRadius,poly_degree,constrained_pi,constrained_v):
    
    #polynomnial coefficients taken based on the degree
    if poly_degree == 0:
        coefficient = 1
    
    if poly_degree == 1:
        coefficient = 4
    
    if poly_degree == 2:
        coefficient = 10
    
    #use only constraint point with non zero weight ( distance < wendlandRadius)
    idxCls = closest_points(xi,constrained_pi,wendlandRadius)
    
    #if the number of constraintt points is less than twice the number of polynomial coefficients
    #assign a large positive outside value to the grid point
    if len(idxCls) < 2 * coefficient:
        return 10000000
    else:
        
        #(B.T * W * B) a(x) = (B.T * W * d)
        
        #B , size n(the number of points with non-zero weight for x 
        #x k(number of coefficient in the basis) from the coordinates of such points
        B = np.zeros((len(idxCls),coefficient))
        
        #Diagonal matrix of size n from the Wendland weights at such points             
        W = np.eye(len(idxCls))
        
        #Vector d of constraint values (0/eps/-eps) corresponding to such points
        d = constrained_v[idxCls]
                     
        p = constrained_pi[idxCls]
        
        for i in range (len(idxCls)):
            
            #W matrix is computed using the wendland function , and the norm. For x,y,z component
            W[i,i] = Wendland(np.linalg.norm(xi-p[i]),wendlandRadius)
            
            #Constraint matrix basis
            B[i] = polyDegree(poly_degree,p[i])
            
        #let's find a(x)
        
        ax = np.linalg.solve(((B.T).dot(W)).dot(B),((B.T).dot(W)).dot(d))
        
        #implicit function
        return polyDegree(poly_degree, xi).dot(ax)

In [27]:
# Generate grid n x n x n

x, T = tet_grid((n, n, n), bbox_min - 0.05 * bbox_diag, bbox_max + 0.05 * bbox_diag)

#Compute implicit sphere function
center = np.array([0., 0., 0.])
print(bbox_diag)
radius = bbox_diag * 0.1
k = 1
#Store the field value fx = f(xi) in a numpy.array, using the same ordering as in x
fx = np.array([MLS(xi,radius,k,constrained_pi,constrained_v) for xi in x])

143.2353308370529


In [28]:
# Treshold fx to visualize inside outside

ind = np.zeros_like(fx)
ind[fx >= 0] =  1
ind[fx < 0] = -1
mp.plot(x, c=ind, shading={"point_size": 4,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Marching to extract surface

In [30]:
# Marcing tet to extract surface

sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
mp.plot(sv, sf, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.6772174…